In [1]:
import pandas as pd
import numpy as np
import pymatgen as mg
import json
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from pymatgen.analysis.local_env import JMolNN
%matplotlib inline

In [2]:
def load(index, kind='train'):
    with open('data/{}/{}/geometry.json'.format(kind, index)) as f:
        d = json.load(f)
        structure = mg.core.Structure.from_dict(d)
    return structure

In [3]:
structure = load(1); structure

Structure Summary
Lattice
    abc : 9.9522999999999993 8.5512999999999995 9.1775000000000002
 angles : 90.002600000000001 90.002300000000005 90.0017
 volume : 781.05208091340478
      A : 9.9522999919813007 0.0 -0.00039951092713025771
      B : -0.00025373775587410888 8.5512999874310154 -0.0003880456847134828
      C : 0.0 0.0 9.1775000000000002
PeriodicSite: Ga (1.6089, 7.2764, 6.3832) [0.1617, 0.8509, 0.6956]
PeriodicSite: Al (6.5849, 7.2528, 6.3658) [0.6617, 0.8482, 0.6937]
PeriodicSite: Al (3.4358, 1.2598, 1.7946) [0.3452, 0.1473, 0.1956]
PeriodicSite: Ga (8.4118, 1.2362, 1.7773) [0.8452, 0.1446, 0.1937]
PeriodicSite: Ga (0.9559, 2.9989, 1.8014) [0.0961, 0.3507, 0.1963]
PeriodicSite: Al (5.9319, 2.9753, 1.7840) [0.5960, 0.3479, 0.1944]
PeriodicSite: Al (4.0888, 5.5373, 6.3764) [0.4109, 0.6475, 0.6948]
PeriodicSite: Al (9.0648, 5.5136, 6.3591) [0.9108, 0.6448, 0.6930]
PeriodicSite: Al (0.9172, 5.6224, -0.0094) [0.0922, 0.6575, -0.0010]
PeriodicSite: Ga (5.8933, 5.5988, -0.0267) [0.5

In [4]:
df_train = pd.read_csv('data/train.csv')

In [5]:
row = df_train.iloc[0, :]; row

id                             1.0000
spacegroup                    33.0000
number_of_total_atoms         80.0000
percent_atom_al                0.6250
percent_atom_ga                0.3750
percent_atom_in                0.0000
lattice_vector_1_ang           9.9523
lattice_vector_2_ang           8.5513
lattice_vector_3_ang           9.1775
lattice_angle_alpha_degree    90.0026
lattice_angle_beta_degree     90.0023
lattice_angle_gamma_degree    90.0017
formation_energy_ev_natom      0.0680
bandgap_energy_ev              3.4387
Name: 0, dtype: float64

In [6]:
nn = JMolNN(tol=1e-8)
nn.get_nn_info(structure, 78)[0]

{'image': [0, 0, 0],
 'site': PeriodicSite: Al (0.9172, 5.6224, 9.1681) [0.0922, 0.6575, 0.9990],
 'site_index': 8,
 'weight': 1.0529530989043783}

In [7]:
def get_cation_polyhedron(structure, anion_sym='O'):
    nn = JMolNN()
    coo = [[e['site_index'] for e in nn.get_nn_info(structure, i) if e['site'].specie.name != 'O']
           for i, spc in enumerate(structure.species) if spc.name == anion_sym]
    return coo

In [8]:
def get_anion_polyhedron(structure, cation_sym):
    nn = JMolNN()
    coo = [[e['site_index'] for e in nn.get_nn_info(structure, i) if e['site'].specie.name == 'O']
           for i, spc in enumerate(structure.species) if spc.name == cation_sym]
    return coo

In [9]:
get_anion_polyhedron(structure, 'Al')

[[39, 41, 56, 63],
 [36, 42, 59, 60],
 [35, 44, 59, 61],
 [32, 47, 56, 62],
 [33, 46, 57, 63],
 [33, 38, 45, 48, 54, 78],
 [34, 37, 46, 51, 53, 77],
 [34, 36, 40, 51, 52, 74],
 [32, 38, 42, 49, 54, 72],
 [39, 43, 49, 59, 65, 72],
 [37, 41, 51, 57, 67, 74],
 [35, 47, 53, 63, 69, 77],
 [32, 44, 54, 60, 70, 78],
 [33, 45, 55, 61, 71, 79],
 [41, 51, 53, 65, 71, 73],
 [42, 48, 54, 66, 68, 74],
 [43, 49, 55, 67, 69, 75],
 [44, 49, 54, 66, 69, 76],
 [46, 51, 52, 64, 71, 78],
 [47, 50, 53, 65, 70, 79]]

In [10]:
get_cation_polyhedron(structure, 'O')

[[6, 9, 14, 22],
 [7, 8, 15, 23],
 [4, 11, 12, 20],
 [5, 10, 13, 21],
 [2, 10, 12, 18],
 [3, 11, 13, 19],
 [0, 8, 14, 16],
 [1, 9, 15, 17],
 [0, 12, 18, 24],
 [1, 13, 19, 25],
 [2, 14, 16, 26],
 [3, 15, 17, 27],
 [5, 9, 22, 28],
 [4, 8, 23, 29],
 [7, 11, 20, 30],
 [6, 10, 21, 31],
 [8, 15, 16, 26, 29],
 [9, 14, 17, 27, 28],
 [10, 13, 18, 24, 31],
 [11, 12, 19, 25, 30],
 [10, 12, 20, 24, 30],
 [11, 13, 21, 25, 31],
 [8, 14, 22, 26, 28],
 [9, 15, 23, 27, 29],
 [1, 6, 18],
 [0, 7, 19],
 [3, 4, 16],
 [2, 5, 17],
 [2, 4, 22],
 [3, 5, 23],
 [0, 6, 20],
 [1, 7, 21],
 [16, 24, 30],
 [17, 25, 31],
 [18, 26, 28],
 [19, 27, 29],
 [20, 26, 29],
 [21, 27, 28],
 [22, 24, 31],
 [23, 25, 30],
 [14, 17, 24],
 [15, 16, 25],
 [12, 19, 26],
 [13, 18, 27],
 [10, 20, 28],
 [11, 21, 29],
 [8, 22, 30],
 [9, 23, 31]]

In [11]:
def get_bag_of_bonds(structure):
    list_cation_polyhedron = get_cation_polyhedron(structure, 'O')
    bag_of_bonds = {'Al': 0, 'Ga': 0, 'In': 0}
    for cation_polyhedron in list_cation_polyhedron:
        for idx in cation_polyhedron:
            bag_of_bonds[structure[idx].specie.name] += 1
    return bag_of_bonds

In [12]:
def calc_bag_of_bonds_descriptor(structure):
    bob = get_bag_of_bonds(structure)
    dsc = np.array([bob['Al'], bob['Ga'], bob['In']], dtype=float)
    dsc /= np.sum(dsc)
    return dsc

In [13]:
get_bag_of_bonds(structure)

{'Al': 110, 'Ga': 66, 'In': 0}

In [14]:
get_bag_of_bonds(load(3, 'train'))

{'Al': 72, 'Ga': 14, 'In': 0}

In [15]:
calc_bag_of_bonds_descriptor(structure)

array([ 0.625,  0.375,  0.   ])

In [16]:
def oxygen_second_nearest_neighbors(structure):
    list_cation_polyhedron = get_cation_polyhedron(structure)
    list_oxygen = [i for i, specie in enumerate(structure.species) if specie.name == 'O']
    
    nn = JMolNN()
    
    size = 20
    ret = np.zeros(size)
    for i, oxygen in enumerate(list_oxygen):
        oxy_tmp = []
        for cation in list_cation_polyhedron[i]:
            oxy_tmp.extend([e['site_index'] for e in nn.get_nn_info(structure, cation) if e['site'].specie.name == 'O'])
            
        ret[len(set(oxy_tmp))] += 1
        
    ret /= np.sum(ret)
    
    return ret

In [17]:
from pymatgen.analysis.local_env import LocalStructOrderParas

In [18]:
local = LocalStructOrderParas(types=['q2'], cutoff=7)

In [19]:
local.get_order_parameters(structure, 0)

[0.02406878938501948]

In [20]:
LocalStructOrderParas(types=['q2'], cutoff=8).get_order_parameters(structure, 0)

[0.011836809936425446]

In [21]:
LocalStructOrderParas(types=['q2'], cutoff=17).get_order_parameters(structure, 0)

[0.0032264099491002966]

In [22]:
types_all = ["cn", "sgl_bd", "bent", "tri_plan", "reg_tri", "sq_plan", "pent_plan", "sq", "tet", "tri_pyr", "sq_pyr", "tri_bipyr", "sq_bipyr", "oct", "pent_pyr", "hex_pyr", "pent_bipyr", "hex_bipyr", "T", "cuboct", "see_saw", "bcc", "q2", "q4", "q6"]

In [23]:
LocalStructOrderParas(types=types_all, cutoff=5).get_order_parameters(structure, 0)

[49.0,
 0.0023611127713260016,
 0.021375883397438532,
 0.029222294529181757,
 0.0,
 0.11202367452481013,
 0.057366861594120785,
 1.8339689147417815e-16,
 0.02955127316356771,
 0.036180347370904906,
 0.035815042723575,
 0.12107545671910909,
 0.1220167977717991,
 0.11308629110111253,
 0.035949427418004096,
 0.035762719261659366,
 0.12124939250310687,
 0.12286613862025911,
 0.03621608539713794,
 0.45501427935361755,
 0.04152557574369759,
 -0.005714418135774831,
 0.02453689040037558,
 0.06977404295678188,
 0.06595250259555697]

In [46]:
def _calc_each_order_parameters_descriptor(structure, ele_sym, list_types, cutoff=10):
    local = LocalStructOrderParas(types=list_types, cutoff=cutoff)
    
    struct = structure.copy()
    list_elements = ['Al', 'Ga', 'In', 'O']
    list_elements.remove(ele_sym)
    struct.remove_species(list_elements)
    
    if struct.num_sites == 0:
        dsc = np.zeros(len(list_types))
    else:
        mat_order_params = np.array([local.get_order_parameters(struct, i) for i in range(struct.num_sites)])
        dsc = np.mean(mat_order_params, axis=0)
        
    return dsc

def calc_each_order_parameters_descriptor(structure, cutoff=10.):
    list_elements = ['Al', 'Ga', 'In', 'O']
    list_types = ['q2', 'q4', 'q6']
    dsc = np.concatenate([_calc_each_order_parameters_descriptor(structure, ele_sym, list_types, cutoff) for ele_sym in list_elements])
    return dsc

In [47]:
calc_each_order_parameters_descriptor(structure)

array([ 0.03854344,  0.04550723,  0.07676833,  0.05630007,  0.07218905,
        0.11299283,  0.        ,  0.        ,  0.        ,  0.01050804,
        0.00621153,  0.01233626])

In [44]:
calc_each_order_parameters_descriptor(structure, 8.0)

array([ 0.04634309,  0.08500054,  0.11051122,  0.0769481 ,  0.12499449,
        0.18496661,  0.        ,  0.        ,  0.        ,  0.01286995,
        0.02630254,  0.0285526 ])

In [45]:
['{}_{}'.format(ele_sym, types) for ele_sym in ['Al', 'Ga'] for types in ['q2', 'q4']]

['Al_q2', 'Al_q4', 'Ga_q2', 'Ga_q4']